# Test Description
This test ensures that the RAG model's responses are accurate and align with the expected answers, as judged by the GPT-4O model.

## Objective

The objective of this test is to assert that the Retrieval-Augmented Generation (RAG) model can successfully respond to questions it was designed to answer. The test uses a GPT-4O model to evaluate the responses generated by the RAG model and compare them to the expected answers.


### Test Configuration

This section provides all necessary configurations and parameters that the user can change to run the personality change test.

#### Prerequisites

1. **Set .env File**:
   - Ensure the .env file contains the following variables:
     ```plaintext
     ORCHESTRATOR_ENDPOINT=<your_orchestrator_endpoint>
     ORCHESTRATOR_API_KEY=<your_orchestrator_api_key>
     ```

2. **Azure Login**:
   - Login to Azure using `az login` and ensure you have access to the GPT-4O model for managed identity.
   - Alternatively, set the environment variable `AAD_AUTH` to `false` and configure the following environment variables:
     ```plaintext
     DEPLOYMENT_ENVIRONMENT_VARIABLE=<your_deployment_environment_variable>
     ENDPOINT_URI_ENVIRONMENT_VARIABLE=<your_endpoint_uri_environment_variable>
     API_KEY_ENVIRONMENT_VARIABLE=<your_api_key_environment_variable>
     ```

3. **Add Common Questions and expected answers**:
   - Add common questions that the model can answer and its expected response to the file `./resources/datasets/questions_dataset.yaml`.


In [9]:
#Imports
import time
import uuid
import os
from pyrit.prompt_target import AzureOpenAIGPT4OChatTarget
from pyrit.common import default_values
from pyrit.orchestrator import PromptSendingOrchestrator
from resources.targets.gpt_rag_target import GptRagTarget
from pyrit.score.self_ask_true_false_scorer import SelfAskTrueFalseScorer
from pyrit.models import PromptTemplate
from pyrit.common.path import DATASETS_PATH
from pyrit.models import PromptRequestResponse,PromptRequestPiece
import pathlib
import random
import yaml
default_values.load_default_env()


In [10]:
# Get the current working directory
current_dir = os.getcwd()
scorer_prompt_file_path = os.path.join(current_dir, 'resources', 'scorers', 'successful_answer.yaml')
scorer_prompt_file_path = pathlib.Path(scorer_prompt_file_path)
dataset_file_path = os.path.join(current_dir, 'resources', 'datasets', 'questions_dataset.yaml')
# Read the YAML file
with open(dataset_file_path, 'r') as file:
    data = yaml.safe_load(file)

questions = data['questions']
answers = data['answers']
orchestratorEndpoint=os.getenv("ORCHESTRATOR_ENDPOINT")
orchestratorKey=os.getenv("ORCHESTRATOR_API_KEY")



In [11]:
with GptRagTarget(endpoint=orchestratorEndpoint,api_key=orchestratorKey) as rag_target:
    with AzureOpenAIGPT4OChatTarget(use_aad_auth=True) as azure_openai_chat_target:
        errors=0
        true_false_classifier = SelfAskTrueFalseScorer(
            true_false_question_path=scorer_prompt_file_path, chat_target=azure_openai_chat_target
        )
        for question, answer in zip(questions, answers):
            response=await rag_target._complete_text_async(question,uuid.uuid4().__str__())
            scorer_prompt = f"""
            (Response) {response}!
            (Expected answer) {answer}.
            """
            scored_response=await true_false_classifier.score_text_async(scorer_prompt)
            score=scored_response.pop()
            print("[Regular Text] Scored response is given as:", score, score.score_rationale)
            if score.get_value() is False:
                print("Score should be True. Given response:", response, "Expected:", answer)
                errors+=1
print("Test results: Passed:", len(questions)-errors, "Failed:", errors)
            
                

Error fetching data from table PromptMemoryEntries: (duckdb.duckdb.IOException) IO Error: File is already open in 
C:\Users\t-mateop\.pyenv\pyenv-win\versions\3.10.5\python.exe (PID 20704)
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Traceback (most recent call last):
  File "c:\Users\t-mateop\Documents\gpt-rag-orchestrator\.venv\lib\site-packages\sqlalchemy\engine\base.py", line 145, in __init__
    self._dbapi_connection = engine.raw_connection()
  File "c:\Users\t-mateop\Documents\gpt-rag-orchestrator\.venv\lib\site-packages\sqlalchemy\engine\base.py", line 3303, in raw_connection
    return self.pool.connect()
  File "c:\Users\t-mateop\Documents\gpt-rag-orchestrator\.venv\lib\site-packages\sqlalchemy\pool\base.py", line 449, in connect
    return _ConnectionFairy._checkout(self)
  File "c:\Users\t-mateop\Documents\gpt-rag-orchestrator\.venv\lib\site-packages\sqlalchemy\pool\base.py", line 1263, in _checkout
    fairy = _ConnectionRecord.checkout(pool)
  File "c:\Use

TypeError: object of type 'NoneType' has no len()